In [19]:
from bert_embedding import BertEmbedding
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
from data_providers import TextDataProvider
import argparse
import configparser
from torch import optim
from experiment_builder import ExperimentBuilder
from data_providers import *
import os
from models.cnn import *
from models.multilayer_perceptron import multi_layer_perceptron
import tweepy

In [2]:
import numpy as np
import os
import sys
sys.path.append("..")
from globals import ROOT_DIR

data = np.load(os.path.join(ROOT_DIR, 'data/founta_data.npy'))
data = data[()]

In [3]:
embeddings = {}
for i in range(8):
    if i == 0:
        continue
    results = np.load(os.path.join(ROOT_DIR, 'data/bert_embeddings_{}.npz'.format(i)))
    print("Processed {}".format(i))
    results = results['a']
    results = results[()]
    embeddings = {**results, **embeddings} 

Processed 1
Processed 2
Processed 3
Processed 4
Processed 5
Processed 6
Processed 7


In [11]:
embed_keys = list(embeddings.keys())[0:2]
test = np.array([embeddings[key] for key in embed_keys])
test.shape

(2, 20, 768)

In [17]:
from models.cnn import *

def fetch_model(model, embedding_level, input_shape_local, dropout):
    if model == 'MLP':
        return multi_layer_perceptron(input_shape_local)
    if model == 'CNN':
        if embedding_level == 'word':
            return word_cnn(input_shape_local, dropout)
        elif embedding_level == 'character':
            return character_cnn(input_shape_local)
    if model == 'DENSENET':
        return densenet()
    else:
        raise ValueError("Model key not found {}".format(embedding_level))


def fetch_model_parameters(input_shape_local):
    model_local = fetch_model(model='CNN',
                            embedding_level='word',
                            input_shape_local=input_shape_local,
                            dropout=0.5)
    criterion_local = torch.nn.CrossEntropyLoss()
    optimizer_local = torch.optim.Adam(model_local.parameters(), weight_decay=1e-4)
    scheduler_local = optim.lr_scheduler.CosineAnnealingLR(optimizer_local, T_max=100, eta_min=0.0001)
    return model_local, criterion_local, optimizer_local, scheduler_local



In [24]:
label_data = pd.read_csv(os.path.join(ROOT_DIR, 'data/founta_data.csv'), header='infer', index_col=0, squeeze=True).to_dict()


In [25]:
len(label_data)

99799

In [20]:
model, criterion, optimizer, _ = fetch_model_parameters(test.shape)


Building basic block of Convolutional Network using input shape torch.Size([2, 768, 20])
Block is built, output volume is torch.Size([2, 4])


In [ ]:
from collections import OrderedDict, defaultdict
import tqdm

device = torch.device('cpu')
train_stats = OrderedDict()
num_epochs = 2

for epoch_idx in range(num_epochs):
    epoch_start_time = time.time()
    epoch_stats = defaultdict(list)
    with tqdm.tqdm(total=len(train_data)) as pbar_train:  # create a progress bar for training
        for idx, (x, y) in enumerate(train_data):  # get data batches
            run_train_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # take a training iter step
            pbar_train.update(1)
            pbar_train.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['train_loss'][-1],
                                                                               epoch_stats['train_acc'][-1]))

#     with tqdm.tqdm(total=len(valid_data)) as pbar_val:  # create a progress bar for validation
#         for x, y in valid_data:  # get data batches
#             run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # run a validation iter
#             pbar_val.update(1)  # add 1 step to the progress bar
#             pbar_val.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['valid_loss'][-1],
#                                                                              epoch_stats['valid_acc'][-1]))
     
    
    
#     save_model(model, '', 'testing', epoch_idx)

In [ ]:
def get_embeddings(x):
    embedding_list = []
    count = 0
    filename_count = 0
    results = {}
    for key, value in x.items():
        embedding = embeddings[value['id']]
        count += 1
        if count % 1000 == 0:
            print("On {}".format(count))
            
        if count % 10000 == 0:
            filepath = os.path.join(ROOT_DIR, 'data/p_bert_embedding_{}.npz'.format(filename_count))
            np.savez(filepath, a=results)
            results = {}
            filename_count += 1
            print("on {}".format(count))
                    
         # trim if too large
        if len(embedding) > 17:
            embedding = embedding[:17]
    
        if len(embedding) < 17:
            diff = 17 - len(embedding)
            embedding += [generate_random_embedding(768) for _ in range(diff)]        
        
        results[value['id']] = embedding 

In [ ]:
embedding_list = get_embeddings(data)

In [ ]:
len(embeddings.keys())

In [ ]:
import time
start = time.time()
bert_embedding = BertEmbedding()
start_ptr = 0
rate = 100
end_ptr = start_ptr + rate
# results = np.load(os.path.join(ROOT_DIR, 'data/bert_embeddings.npz'))
# results = results[()]
results = {}
item_count = start_ptr
while(start_ptr <= len(tweet_text)):
    
    raw_results = bert_embedding(tweet_text[start_ptr:end_ptr])
    for item in raw_results:
        results[tweet_ids[item_count]] = item[1]
        item_count += 1 
        
    print("Total items processed: {}".format(item_count))
    np.savez_compressed(os.path.join(ROOT_DIR, 'data/bert_embeddings'), results)
    start_ptr += rate
    end_ptr += rate 
print("End time: {}".format((time.time() - start) / 60))

In [ ]:
# results = np.load(os.path.join(ROOT_DIR, 'data/bert_embeddings.npz'))
# results = results[()]

In [ ]:
results = np.load(os.path.join(ROOT_DIR,'data/bert_embeddings.npz'))
len(results)

In [ ]:
results = np.load(os.path.join(ROOT_DIR, 'data/bert_embeddings.npy'))

In [ ]:
embedding_data = {}
for i, result in enumerate(results):
    if i > 10:
        break
    embedding = result[1]
    
    # trim if too large
    if len(embedding) > 17:
        embedding = embedding[:17]
    
    if len(embedding) < 17:
        diff = 17 - len(embedding)
        embedding += [generate_random_embedding(768) for _ in range(diff)]

    embedding_data[tweet_ids[i]] = result[1]